<a href="https://colab.research.google.com/github/Rodrigo-Henrique21/ProjetoEngenhariaMTTA/blob/main/API_TransporteEUA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Criando Sessão Spark e Tunel***

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 36.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=cc9eb77becccfc813925be95600e2c06814d26a1c38c53d3bcfa9993ee6ae6e0
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok authtoken 2FmnG5PC0aPutLRSrgM3XoiWe2s_2i27zKtF1hnEmUygma9ts')
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [ ]:
spark

# ***Importando Bibliotecas***

In [ ]:
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install azure-storage-blob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 383 kB 4.3 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 4.0 MB 47.6 MB/s 
     |████████████████████████████████| 178 kB 56.4 MB/s 
     |████████████████████████████████| 41 kB 470 kB/s 


In [ ]:
spark.conf.set( 
  "fs.azure.account.key.datalakegin2.dfs.core.windows.net","/3jB2icleilbB3TjypeN2VqyNze7p3lEA0s7bFJUk7sTBPk+KZScqSM0JdWEt7rn9TihQsP3MzWB+AStSvVWqA==")

In [ ]:
spark.conf.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")

In [ ]:
import datetime as dt
import os
import json
import requests
import time as time
import pandas as pd
import os
import pathlib
from os import walk
from azure.storage.blob import PublicAccess, ContainerClient, BlobServiceClient, BlobClient
from pyspark.sql.types import StringType,IntegerType,DateType,TimestampType
from pyspark.sql import types
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [ ]:
Key = '7e33d7e4973646048483efb47ef4c84c'
api_inicio = 'https://api-v3.mbta.com'
api_parametro = '/alerts?'
api_chave = f'&api_key={Key}'

# ***Realizando coleta dos dados***

In [ ]:
api = api_inicio+api_parametro+api_chave
chamada = requests.get(api).status_code


In [ ]:
if (chamada == 200):
  print('Conversão em andamento..')
  chamada = requests.get(api)
  chamada = chamada.json()
  time.sleep(2)
  print('Conversão finalizada')
else:
  print(str(chamada.status_code))


Conversão em andamento..
Conversão finalizada


In [ ]:
chamada = chamada['data']

In [ ]:
chamada

[{'attributes': {'active_period': [{'end': None,
     'start': '2022-09-21T16:37:00-04:00'}],
   'banner': None,
   'cause': 'MAINTENANCE',
   'created_at': '2022-09-21T16:37:32-04:00',
   'description': None,
   'effect': 'ESCALATOR_CLOSURE',
   'header': 'Wellington Escalator 310 (Oak Grove platform to paid lobby) unavailable due to maintenance',
   'informed_entity': [{'activities': ['USING_ESCALATOR'],
     'facility': '310',
     'stop': '70033'},
    {'activities': ['USING_ESCALATOR'],
     'facility': '310',
     'stop': 'place-welln'}],
   'lifecycle': 'ONGOING',
   'service_effect': 'Wellington escalator unavailable',
   'severity': 3,
   'short_header': 'Wellington Escalator 310 (Oak Grove platform to paid lobby) unavailable due to maintenance',
   'timeframe': 'ongoing',
   'updated_at': '2022-10-20T15:04:55-04:00',
   'url': None},
  'id': '463481',
  'links': {'self': '/alerts/463481'},
  'type': 'alert'},
 {'attributes': {'active_period': [{'end': None,
     'start': '202

In [ ]:
banner = []
cause = [] 
created = []
description = [] 
effect = []
header = []
informed_entity = []
informed_entity_stop = []
informed_entity_route = []
informed_entity_route_Type = []
informed_entity_activities = []
lifecycle = []
service_effect = []
severity = []
links = []
short_header = []
updated_at = []
id = []
links = []


contador = 0
for key in chamada:
  if (contador == 0):
    print('Realizando extração das informações..')
  else:
    banner.append(key['attributes']['banner'])
    cause.append(key['attributes']['cause'])
    created.append(key['attributes']['created_at'])
    description.append(key['attributes']['description'])
    effect.append(key['attributes']['effect'])
    header.append(key['attributes']['header'])
    lifecycle.append(key['attributes']['lifecycle'])
    informed_entity.append(key['attributes']['informed_entity'])
    service_effect.append(key['attributes']['service_effect'])
    severity.append(key['attributes']['severity'])
    short_header.append(key['attributes']['short_header'])
    updated_at.append(key['attributes']['updated_at'])
    links.append(key['links']['self'])
    for active in key['attributes']['informed_entity']:
        if (len(active['activities']) == 1):
          informed_entity_activities.append(active['activities'])
        else:
          informed_entity_activities.append('-')
        if ("stop" in active.keys()):
          informed_entity_stop.append(active['stop'])
        elif ("route" in active.keys()):
          informed_entity_route.append(active['route'])
        elif ("route_type" in active.keys()):
          informed_entity_route_Type.append(active['route_type'])
        elif ("facility" in active.keys()):
          informed_entity_route.append(active['facility'])
        elif ("trip" in active.keys()):
          informed_entity_stop.append(active['trip'])
  contador += 1


Realizando extração das informações..


In [ ]:
active_period_inicio = []
active_period_fim = []

for key in chamada:
  for data in key['attributes']['active_period']:
      active_period_inicio.append(data['start'])
      active_period_fim.append(data['end'])


In [ ]:
print(len(cause))
print(len(banner))
print(len(header))
print(len(service_effect))
print(len(created))
print(len(informed_entity))
print(len(informed_entity_stop))

187
187
187
187
187
187
639


In [ ]:
alertas = {
    'banner':banner,
    'cause':cause,
    'created':created,
    'description':description,
    'effect':effect,
    'header':header,
    'lifecycle':lifecycle,
    'severity':severity,
    'informed_entity':informed_entity,
    'service_effect':service_effect,
    'short_header':short_header,
    'updated_at':updated_at,
    'links':links
}


# ***Passando para DataFrame***

In [ ]:
alerta = pd.DataFrame(alertas)

In [ ]:
total_linhas = alerta.count()
total_linhas

banner               0
cause              187
created            187
description         74
effect             187
header             187
lifecycle          187
severity           187
informed_entity    187
service_effect     187
short_header       187
updated_at         187
links              187
dtype: int64

In [ ]:
alerta.head(100)

,banner,cause,created,description,effect,header,lifecycle,severity,informed_entity,service_effect,short_header,updated_at,links
0,None,MAINTENANCE,2022-09-21T16:39:32-04:00,None,ESCALATOR_CLOSURE,Sullivan Square Escalator 308 (Oak Grove platf...,ONGOING,3,"[{'activities': ['USING_ESCALATOR'], 'facility...",Sullivan Square escalator unavailable,Sullivan Square Escalator 308 (Oak Grove platf...,2022-10-20T15:06:04-04:00,/alerts/463485
1,None,TRAFFIC,2022-10-20T17:49:18-04:00,None,DELAY,Route 22 experiencing delays of up to 20 minut...,NEW,5,"[{'activities': ['BOARD', 'EXIT', 'RIDE'], 'ro...",Route 22 delay,Route 22 experiencing delays of up to 20 minut...,2022-10-20T17:49:18-04:00,/alerts/468439
2,None,MAINTENANCE,2022-10-20T09:29:42-04:00,"To exit, exit the train at Tufts Mediical Cent...",ELEVATOR_CLOSURE,Chinatown Elevator 922 (Forest Hills platform ...,UPCOMING,3,"[{'activities': ['USING_WHEELCHAIR'], 'facilit...",Chinatown elevator unavailable,Chinatown Elevator 922 (Forest Hills platform ...,2022-10-20T14:17:26-04:00,/alerts/468340
3,None,UNKNOWN_CAUSE,2022-10-20T05:14:30-04:00,Affected stops:\r\nBrandeis/Roberts,TRACK_CHANGE,Please board all Fitchburg Line trains on the ...,NEW,1,"[{'activities': ['BOARD'], 'route': 'CR-Fitchb...",Fitchburg Line track change,Please board all Fitchburg Line trains on the ...,2022-10-20T05:14:30-04:00,/alerts/468278
4,None,UNKNOWN_CAUSE,2022-07-25T12:37:22-04:00,None,SERVICE_CHANGE,"CT3, 8 and 47 notice: The stop at 88 E Newton ...",ONGOING,3,"[{'activities': ['BOARD', 'EXIT', 'RIDE'], 'ro...",Bus notice,"CT3, 8 and 47 notice: The stop at 88 E Newton ...",2022-10-19T06:44:00-04:00,/alerts/454798
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,UNKNOWN_CAUSE,2022-08-10T07:15:17-04:00,None,SERVICE_CHANGE,Route 87 and 88 notice: The stop at Holland St...,ONGOING,3,"[{'activities': ['BOARD', 'EXIT', 'RIDE'], 'ro...",Routes 87 and 88 notice,Rt. 87 & 88: The stop at Holland St @ Moore St...,2022-10-19T06:26:09-04:00,/alerts/457130
96,None,CONSTRUCTION,2022-05-24T15:32:59-04:00,Affected stops:\r\nRiver St @ Hyde Park Ave (o...,DETOUR,Route 33 detoured due to construction. Inbound...,ONGOING,7,"[{'activities': ['BOARD', 'EXIT'], 'route': '3...",Route 33 detour,Rt. 33 detoured. Inb. connect at Reservation R...,2022-05-31T15:33:16-04:00,/alerts/445295
97,None,UNKNOWN_CAUSE,2022-07-19T07:29:43-04:00,Affected direction: Inbound\r\n\r\nAffected st...,DETOUR,Route 556 inbound is bypassing 184 Hammond St ...,ONGOING_UPCOMING,7,"[{'activities': ['BOARD', 'EXIT'], 'direction_...",Route 556 detour,Route 556 inbound is bypassing 184 Hammond St ...,2022-08-02T07:30:15-04:00,/alerts/453816
98,None,POLICE_ACTION,2022-07-19T12:55:42-04:00,Affected stops:\r\nSouth Bay Mall @ Allstate R...,DETOUR,Route 8 detoured due to construction. Nearest ...,ONGOING,7,"[{'activities': ['BOARD', 'EXIT'], 'route': '8...",Route 8 detour,Route 8 detoured due to construction. Nearest ...,2022-07-26T12:56:16-04:00,/alerts/453892


# ***Salvando na camada Raw***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def Upload_file(container_name,filename):
  connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakegin2;AccountKey=/3jB2icleilbB3TjypeN2VqyNze7p3lEA0s7bFJUk7sTBPk+KZScqSM0JdWEt7rn9TihQsP3MzWB+AStSvVWqA==;EndpointSuffix=core.windows.net'
  container_name = container_name
  filename = filename 
  container_client = BlobServiceClient.from_connection_string(connection_string)
  print(f'Seu container{container_client}')
  print()
  print(f'Realizando upload do arquivo {filename}')
  blob_client = container_client.get_blob_client(container=container_name, blob = filename)   
  output = pd.DataFrame(alerta).to_json()
  blob_client.upload_blob(output, blob_type="BlockBlob", overwrite = True)
  print(f'Upload realizado com sucesso..')

def download_blob(blob_client, blob_name):
    print('[{}]:[INFO] : Blob name: {}'.format(dt.datetime.utcnow(), blob_name))
    print("[{}]:[INFO] : Downloading {} ...".format(dt.datetime.utcnow(), blob_name))
    blob_data = blob_client.download_blob()
    print("[{}]:[INFO] : download finished".format(dt.datetime.utcnow()))
    return blob_data

def download_file_container(download_file_name):
  connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakegin2;AccountKey=/3jB2icleilbB3TjypeN2VqyNze7p3lEA0s7bFJUk7sTBPk+KZScqSM0JdWEt7rn9TihQsP3MzWB+AStSvVWqA==;EndpointSuffix=core.windows.net'
  container_name = 'raw'
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)
  container_client = blob_service_client.get_container_client(container_name)
  blob_list = container_client.list_blobs()
  for blob in blob_list:
    if blob.name == download_file_name:
      blob_client = container_client.get_blob_client(blob.name)
      file_silver = download_blob(blob_client, blob.name)
  return file_silver

In [ ]:
Upload_file('raw','alerta.json')

Seu container<azure.storage.blob._blob_service_client.BlobServiceClient object at 0x7f88cd7e4610>

Realizando upload do arquivo alerta.json
Upload realizado com sucesso..


In [ ]:
alerta.to_parquet('/content/drive/MyDrive/Treinamento Spark_3/API Transportes/Alertas/alerta.pq')

In [ ]:
alerta.columns

Index(['banner', 'cause', 'created', 'description', 'effect', 'header',
       'lifecycle', 'severity', 'informed_entity', 'service_effect',
       'short_header', 'updated_at', 'links'],
      dtype='object')

# ***Baixando da Raw e lendo com Spark***

In [ ]:
file_silver = download_file_container('alerta.json')

[2022-10-20 22:04:58.698592]:[INFO] : Blob name: alerta.json
[2022-10-20 22:04:58.699518]:[INFO] : Downloading alerta.json ...
[2022-10-20 22:05:00.252445]:[INFO] : download finished


In [ ]:
print(file_silver)

In [ ]:
df = pd.read_json(file_silver.content_as_bytes())

In [ ]:
alerta_silver = spark.createDataFrame(df)

# ***Começando tratamento dos dados para silver***

In [ ]:
alerta_silver.limit(100).show()

+------+-------------+--------------------+--------------------+-----------------+--------------------+---------+--------+--------------------+--------------------+--------------------+--------------------+--------------+
|banner|        cause|             created|         description|           effect|              header|lifecycle|severity|     informed_entity|      service_effect|        short_header|          updated_at|         links|
+------+-------------+--------------------+--------------------+-----------------+--------------------+---------+--------+--------------------+--------------------+--------------------+--------------------+--------------+
|   NaN|  MAINTENANCE|2022-09-21T16:39:...|                null|ESCALATOR_CLOSURE|Sullivan Square E...|  ONGOING|       3|[{stop -> null, f...|Sullivan Square e...|Sullivan Square E...|2022-10-20T15:06:...|/alerts/463485|
|   NaN|      TRAFFIC|2022-10-20T17:49:...|                null|            DELAY|Route 22 experien...|      NEW

In [ ]:
alerta_silver.printSchema()

root
 |-- banner: double (nullable = true)
 |-- cause: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- effect: string (nullable = true)
 |-- header: string (nullable = true)
 |-- lifecycle: string (nullable = true)
 |-- severity: long (nullable = true)
 |-- informed_entity: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: array (valueContainsNull = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- service_effect: string (nullable = true)
 |-- short_header: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- links: string (nullable = true)



In [ ]:
colunas_new = ['Bandeira','Causa','Criado','Descricao','efeito','cabecalho','CicloDeVida','Gravidade','informativo','Servico_Efeito','Cabecalho_curto','atualizado','links']

In [ ]:
colunas_df = alerta.columns
colunas_old = []
for col in colunas_df:
  colunas_old.append(col)
colunas_old

['banner',
 'cause',
 'created',
 'description',
 'effect',
 'header',
 'lifecycle',
 'severity',
 'informed_entity',
 'service_effect',
 'short_header',
 'updated_at',
 'links']

In [ ]:
for old,new in zip(colunas_old,colunas_new):
  alerta_silver = alerta_silver.withColumnRenamed(old,new)

In [ ]:
#alerta_silver = alerta_silver.withColumn('Data_Inicio',F.regexp_replace(alerta_silver['Data_Inicio'], 'T',' '))
#alerta_silver = alerta_silver.withColumn('Data_Fim',F.regexp_replace(alerta_silver['Data_Fim'], 'T',' '))

In [ ]:
#alerta_silver = alerta_silver.withColumn('Data_Inicio',F.substring(alerta_silver['Data_Inicio'],0,19))
#alerta_silver = alerta_silver.withColumn('Data_Fim',F.substring(alerta_silver['Data_Fim'], 0,19))

In [ ]:
alerta_silver.show()

+--------+-------------+--------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+--------------------+--------------+
|Bandeira|        Causa|              Criado|           Descricao|           efeito|           cabecalho|CicloDeVida|Gravidade|         informativo|      Servico_Efeito|     Cabecalho_curto|          atualizado|         links|
+--------+-------------+--------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+--------------------+--------------+
|     NaN|  MAINTENANCE|2022-09-21T16:39:...|                null|ESCALATOR_CLOSURE|Sullivan Square E...|    ONGOING|        3|[{stop -> null, f...|Sullivan Square e...|Sullivan Square E...|2022-10-20T15:06:...|/alerts/463485|
|     NaN|      TRAFFIC|2022-10-20T17:49:...|                null|            DELAY|Route 22

In [ ]:
alerta_silver = alerta_silver.withColumn('Bandeira',alerta_silver['Bandeira'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Causa',alerta_silver['Causa'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Descricao',alerta_silver['Descricao'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('efeito',alerta_silver['efeito'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('cabecalho',alerta_silver['cabecalho'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('CicloDeVida',alerta_silver['CicloDeVida'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Servico_Efeito',alerta_silver['Servico_Efeito'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Cabecalho_curto',alerta_silver['Cabecalho_curto'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('links',alerta_silver['links'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Gravidade',alerta_silver['Gravidade'].cast(IntegerType()))


In [ ]:
alerta_silver = alerta_silver.withColumn('Links',F.substring(alerta_silver['Links'],9,14).alias('Links'))
alerta_silver = alerta_silver.withColumnRenamed('Links','Id_Alerta')
alerta_silver = alerta_silver.withColumn('Id_Alerta',alerta_silver['Id_Alerta'].cast(StringType()))

In [ ]:
#alerta_silver = alerta_silver.withColumn('Data_Inicio', F.unix_timestamp('Data_Inicio',"yyyy-MM-dd HH:mm:ss").cast(TimestampType()))
#alerta_silver = alerta_silver.withColumn('Data_Fim', F.unix_timestamp('Data_Fim',"yyyy-MM-dd HH:mm:ss").cast(TimestampType()))

In [ ]:
alerta_silver.printSchema()

root
 |-- Bandeira: string (nullable = true)
 |-- Causa: string (nullable = true)
 |-- Criado: string (nullable = true)
 |-- Descricao: string (nullable = true)
 |-- efeito: string (nullable = true)
 |-- cabecalho: string (nullable = true)
 |-- CicloDeVida: string (nullable = true)
 |-- Gravidade: integer (nullable = true)
 |-- informativo: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: array (valueContainsNull = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- Servico_Efeito: string (nullable = true)
 |-- Cabecalho_curto: string (nullable = true)
 |-- atualizado: string (nullable = true)
 |-- Id_Alerta: string (nullable = true)



In [ ]:
alerta_silver.show()

+--------+-------------+--------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+--------------------+---------+
|Bandeira|        Causa|              Criado|           Descricao|           efeito|           cabecalho|CicloDeVida|Gravidade|         informativo|      Servico_Efeito|     Cabecalho_curto|          atualizado|Id_Alerta|
+--------+-------------+--------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+--------------------+---------+
|     NaN|  MAINTENANCE|2022-09-21T16:39:...|                null|ESCALATOR_CLOSURE|Sullivan Square E...|    ONGOING|        3|[{stop -> null, f...|Sullivan Square e...|Sullivan Square E...|2022-10-20T15:06:...|   463485|
|     NaN|      TRAFFIC|2022-10-20T17:49:...|                null|            DELAY|Route 22 experien...|       

In [ ]:
alerta_silver = alerta_silver.withColumn('Criado',F.regexp_replace('Criado','T',' ').alias('Criado'))
alerta_silver = alerta_silver.withColumn('Criado',F.substring('Criado',0,19).alias('Criado'))
alerta_silver = alerta_silver.withColumn('Criado',F.unix_timestamp('Criado','yyyy-MM-dd HH:mm:ss').cast(TimestampType()))

In [ ]:
alerta_silver = alerta_silver.withColumn('Atualizado',F.regexp_replace('Atualizado','T',' ').alias('Atualizado'))
alerta_silver = alerta_silver.withColumn('Atualizado',F.substring('Atualizado',0,19).alias('Atualizado'))
alerta_silver = alerta_silver.withColumn('Atualizado',F.unix_timestamp('Atualizado','yyyy-MM-dd HH:mm:ss').cast(TimestampType()))

In [ ]:
alerta_silver.show()

+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+-------------------+---------+
|Bandeira|        Causa|             Criado|           Descricao|           efeito|           cabecalho|CicloDeVida|Gravidade|         informativo|      Servico_Efeito|     Cabecalho_curto|         Atualizado|Id_Alerta|
+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+-------------------+---------+
|     NaN|  MAINTENANCE|2022-09-21 16:39:32|                null|ESCALATOR_CLOSURE|Sullivan Square E...|    ONGOING|        3|[{stop -> null, f...|Sullivan Square e...|Sullivan Square E...|2022-10-20 15:06:04|   463485|
|     NaN|      TRAFFIC|2022-10-20 17:49:18|                null|            DELAY|Route 22 experien...|        NEW|    

In [ ]:
alerta_silver = alerta_silver.withColumn('informativo',F.explode('informativo'))
alerta_silver = alerta_silver.withColumn('Stop',alerta_silver['informativo.stop'])
alerta_silver = alerta_silver.withColumn('Route',alerta_silver['informativo.route'])
alerta_silver = alerta_silver.withColumn('Route_Type',alerta_silver['informativo.route_type'])
alerta_silver = alerta_silver.withColumn('Activities',alerta_silver['informativo.activities'])
alerta_silver = alerta_silver.drop('informativo')
alerta_silver.show()

+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+-------------------+---------+----+-----+----------+-------------------+
|Bandeira|        Causa|             Criado|           Descricao|           efeito|           cabecalho|CicloDeVida|Gravidade|      Servico_Efeito|     Cabecalho_curto|         Atualizado|Id_Alerta|Stop|Route|Route_Type|         Activities|
+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+-------------------+---------+----+-----+----------+-------------------+
|     NaN|  MAINTENANCE|2022-09-21 16:39:32|                null|ESCALATOR_CLOSURE|Sullivan Square E...|    ONGOING|        3|Sullivan Square e...|Sullivan Square E...|2022-10-20 15:06:04|   463485|null| null|      null|  [USING_ESCALATOR]|
|     NaN|  MAINTENANCE|2022-09-21 1

In [ ]:
alerta_silver = alerta_silver.withColumn('Stop',alerta_silver['Stop'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Route',alerta_silver['Route'].cast(StringType()))
alerta_silver = alerta_silver.withColumn('Route_Type',alerta_silver['Route_Type'].cast(StringType()))

In [ ]:
alerta_silver = alerta_silver.withColumn('Activities',F.explode('Activities'))

In [ ]:
alerta_silver.printSchema()

root
 |-- Bandeira: string (nullable = true)
 |-- Causa: string (nullable = true)
 |-- Criado: timestamp (nullable = true)
 |-- Descricao: string (nullable = true)
 |-- efeito: string (nullable = true)
 |-- cabecalho: string (nullable = true)
 |-- CicloDeVida: string (nullable = true)
 |-- Gravidade: integer (nullable = true)
 |-- Servico_Efeito: string (nullable = true)
 |-- Cabecalho_curto: string (nullable = true)
 |-- Atualizado: timestamp (nullable = true)
 |-- Id_Alerta: string (nullable = true)
 |-- Stop: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Route_Type: string (nullable = true)
 |-- Activities: string (nullable = true)



In [ ]:
alerta_silver = alerta_silver.na.fill({'Stop':'-','Route':'-','Route_Type':'-'})

In [ ]:
alerta_silver.show()

+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+-------------------+---------+----+-----+----------+----------------+
|Bandeira|        Causa|             Criado|           Descricao|           efeito|           cabecalho|CicloDeVida|Gravidade|      Servico_Efeito|     Cabecalho_curto|         Atualizado|Id_Alerta|Stop|Route|Route_Type|      Activities|
+--------+-------------+-------------------+--------------------+-----------------+--------------------+-----------+---------+--------------------+--------------------+-------------------+---------+----+-----+----------+----------------+
|     NaN|  MAINTENANCE|2022-09-21 16:39:32|                null|ESCALATOR_CLOSURE|Sullivan Square E...|    ONGOING|        3|Sullivan Square e...|Sullivan Square E...|2022-10-20 15:06:04|   463485|   -|    -|         -| USING_ESCALATOR|
|     NaN|  MAINTENANCE|2022-09-21 16:39:32|    

In [ ]:
alerta_silver.coalesce(1).write.mode('overwrite').parquet("/content/drive/MyDrive/Treinamento Spark_3/API Transportes/Alertas/alerta_silver.parquet")

In [ ]:
def Upload_file_silver(container_name,filename,listfiles):
  connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakegin2;AccountKey=/3jB2icleilbB3TjypeN2VqyNze7p3lEA0s7bFJUk7sTBPk+KZScqSM0JdWEt7rn9TihQsP3MzWB+AStSvVWqA==;EndpointSuffix=core.windows.net'
  container_name = container_name
  filename = filename 
  container_client = BlobServiceClient.from_connection_string(connection_string)
  print(f'Seu container{container_client}')
  print()
  print(f'Realizando upload do arquivo {filename}')
  blob_client = container_client.get_blob_client(container=container_name, blob = filename)   
  output = pd.read_parquet(listfiles).to_parquet()
  #output = pd.read_parquet('/content/drive/MyDrive/Treinamento Spark_3/API Transportes/Alertas/alerta_silver.parquet/part-00000-2b6589ab-f67e-458d-b248-0f0474617ba0-c000.snappy.parquet').to_parquet()
  blob_client.upload_blob(output, blob_type="BlockBlob", overwrite = True)
  print(f'Upload realizado com sucesso..')


In [ ]:
path = '/content/drive/MyDrive/Treinamento Spark_3/API Transportes/Alertas/alerta_silver.parquet'
f = []
uploados_realizados = []

for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    for filename in f:
      if (pathlib.Path(filename).suffix == '.parquet'):
        file_complete = os.path.join(path,filename)
        uploados_realizados.append(filename)
        print(f'Realizando upload de {len(uploados_realizados)} arquivo')
        Upload_file_silver('silver','alerta_silver.pq',file_complete)

Realizando upload de 1 arquivo
Seu container<azure.storage.blob._blob_service_client.BlobServiceClient object at 0x7f88cb0db6d0>

Realizando upload do arquivo alerta_silver.pq
Upload realizado com sucesso..


# ***Analise Dos Dados***

In [ ]:
alerta_silver.createOrReplaceTempView('alertaView')

In [ ]:
spark.sql ("""

SELECT Causa As Causa,
       COUNT(Causa) As Total
       FROM alertaView
       GROUP BY Causa
       ORDER BY Total DESC
""").show()

+-------------+-----+
|        Causa|Total|
+-------------+-----+
|UNKNOWN_CAUSE|  867|
| CONSTRUCTION|  674|
|      TRAFFIC|  132|
|  MAINTENANCE|   84|
|       PARADE|   22|
|   TRACK_WORK|    9|
|SPECIAL_EVENT|    4|
|POLICE_ACTION|    4|
|TRAIN_TRAFFIC|    3|
+-------------+-----+



In [ ]:
spark.sql("""

SELECT Id_Alerta As Id_Alerta,
       COUNT(DISTINCT Id_Alerta) As Contagem_Id
       FROM alertaView
       GROUP BY Id_Alerta
       
""").show()

+---------+-----------+
|Id_Alerta|Contagem_Id|
+---------+-----------+
|   466613|          1|
|   468438|          1|
|   384242|          1|
|   460419|          1|
|   390223|          1|
|   384239|          1|
|   468436|          1|
|   468437|          1|
|   468392|          1|
|   468410|          1|
|   468354|          1|
|   468415|          1|
|   468278|          1|
|   468414|          1|
|   391877|          1|
|   468088|          1|
|   468386|          1|
|   462756|          1|
|   468402|          1|
|   463484|          1|
+---------+-----------+
only showing top 20 rows



In [ ]:
spark.sql ("""

SELECT DISTINCT Criado
       FROM alertaView


""").show()

+-------------------+
|             Criado|
+-------------------+
|2022-10-19 06:13:31|
|2022-08-25 08:33:56|
|2022-08-30 16:39:49|
|2022-08-10 06:56:04|
|2022-10-14 11:13:13|
|2022-07-28 11:31:03|
|2022-09-21 16:39:32|
|2022-10-20 16:54:38|
|2022-10-20 17:06:33|
|2022-10-20 17:15:14|
|2022-08-03 19:19:10|
|2022-10-20 09:29:42|
|2022-10-20 16:23:58|
|2022-07-25 12:37:22|
|2021-11-11 07:39:51|
|2022-10-13 13:17:00|
|2022-10-20 15:34:34|
|2022-10-20 17:48:50|
|2022-05-03 09:55:25|
|2022-10-20 15:23:12|
+-------------------+
only showing top 20 rows



In [ ]:
spark.sql ("""

SELECT COUNT(*)
FROM alertaView

""").show()

+--------+
|count(1)|
+--------+
|    1799|
+--------+



In [ ]:
spark.sql("""

SELECT DISTINCT Atualizado,
       Criado,
       Stop,
       Activities
       FROM alertaView
       GROUP BY Activities,Atualizado,Stop,Criado


""").show()

+-------------------+-------------------+----+----------------+
|         Atualizado|             Criado|Stop|      Activities|
+-------------------+-------------------+----+----------------+
|2022-10-19 06:26:52|2022-08-10 06:56:04|   -|            EXIT|
|2022-10-20 16:51:17|2022-10-20 16:51:17|   -|           BOARD|
|2022-10-19 06:44:52|2022-08-30 16:31:35|   -|            EXIT|
|2022-10-17 12:58:38|2022-09-16 14:27:48|   -|            RIDE|
|2022-10-19 06:22:55|2022-08-25 08:33:25|   -|            RIDE|
|2022-10-19 06:32:12|2022-07-28 11:31:03|   -|           BOARD|
|2021-12-15 11:40:09|2021-03-04 10:35:23|   -|            RIDE|
|2022-10-19 06:27:16|2022-08-10 06:55:22|   -|           BOARD|
|2022-10-20 14:21:21|2019-02-01 15:02:13|   -|USING_WHEELCHAIR|
|2022-10-20 15:16:40|2022-10-20 15:16:40|   -|            EXIT|
|2021-03-07 20:33:20|2020-06-11 08:12:41|   -|           BOARD|
|2021-06-13 08:16:49|2021-04-20 14:34:14|   -|           BOARD|
|2022-10-19 06:39:53|2022-07-25 12:33:28